In [1]:
import pandas as pd
out=pd.DataFrame()

#### Telegram
начало - max(дата первой новости, дата первой цены)

конец - min(дата последней новости, дата последней цены)

In [2]:
p = pd.read_csv('C:/Users/79627/WPy64-3741/notebooks/telegram/data/9_join.csv', sep='|')
p.head(3)

,ticker,date,open,high,low,close,vol,volatility_5,volatility_20,volatility_120,...,label_10_1.0,label_10_2.0,label_10_3.0,label_10_4.0,label_10_5.0,label_10_6.0,label_10_7.0,label_10_8.0,label_10_9.0,label_10_10.0
0,zvez,20210127.0,4.240,4.500,4.210,4.395,588000,0.151559,0.122347,0.303120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,zvez,20210119.0,4.145,4.195,4.145,4.155,52000,0.036277,0.061368,0.308242,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,zvez,20201229.0,4.020,4.020,3.945,4.000,87000,0.019131,0.032221,0.304417,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
p = p[['ticker', 'date', 'close']]
p.head()

,ticker,date,close
0,zvez,20210127.0,4.395
1,zvez,20210119.0,4.155
2,zvez,20201229.0,4.000
3,zvez,20201209.0,4.065
4,zvez,20200924.0,4.570


In [4]:
df_min = p[['date', 'ticker']].groupby('ticker').min().rename(columns = {'date':'min_date_price'}).reset_index()
df_min['min_dt'] = df_min.max(axis=1)
df_min = df_min.merge(p[['ticker', 'date', 'close']].rename(columns = {'date':'min_dt'}), how='inner', on=['ticker', 'min_dt'])
df_min.head()


,ticker,min_date_price,min_dt,close
0,abrd,20190610.0,20190610.0,138.50
1,afks,20170929.0,20170929.0,13.36
2,aflt,20171124.0,20171124.0,159.85
3,aflt,20171124.0,20171124.0,159.85
4,agro,20180112.0,20180112.0,582.00


In [5]:
df_max = p[['date', 'ticker']].groupby('ticker').max().rename(columns = {'date':'max_date_price'}).reset_index()
df_max['max_dt'] = df_max.min(axis=1)
df_max = df_max.merge(p[['ticker', 'date', 'close']].rename(columns = {'date':'max_dt'}), how='inner', on=['ticker', 'max_dt'])
df_max

,ticker,max_date_price,max_dt,close
0,abrd,20210310.0,20210310.0,206.000
1,afks,20210302.0,20210302.0,35.194
2,afks,20210302.0,20210302.0,35.194
3,afks,20210302.0,20210302.0,35.194
4,afks,20210302.0,20210302.0,35.194
...,...,...,...,...
167,yndx,20210301.0,20210301.0,5004.600
168,yndx,20210301.0,20210301.0,5004.600
169,yndx,20210301.0,20210301.0,5004.600
170,zill,20210211.0,20210211.0,3230.000


In [6]:
tg = df_max.merge(df_min, how='inner', on='ticker')
tg = tg[['ticker', 'min_dt', 'max_dt', 'close_y', 'close_x']]
tg['bh'] = (tg.close_x / tg.close_y - 1) * 100
tg = tg.drop_duplicates()

In [11]:
tg[['ticker', 'bh']].to_csv('C:/Users/79627/WPy64-3741/notebooks/telegram/data/tg_bh.csv', sep='|', index=False)

In [12]:
tg[['ticker', 'bh']].sort_values('bh').to_excel('C:/Users/79627/WPy64-3741/notebooks/telegram/data/bh.xlsx')